In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

DIR = "/content/drive/My Drive/garbage_classification2"

# Load the entire dataset without splitting into training and validation sets
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DIR,
    batch_size=32,
    image_size=(224, 224)
)

# Extract class names and number of classes
classes = dataset.class_names
numClasses = len(classes)
print(classes, numClasses)

# Load ResNet50 model with pre-trained weights
image_size = (224, 224)
resnet50_model = ResNet50(input_shape=image_size + (3,), weights='imagenet', include_top=False)

# Freeze layers
for layer in resnet50_model.layers:
    layer.trainable = False

# Build ResNet50-based model for classification
inputs = tf.keras.Input(shape=image_size + (3,))
x = tf.keras.applications.resnet50.preprocess_input(inputs)  # Minimal preprocessing
x = resnet50_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(numClasses, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
history = model.fit(dataset, epochs=10)

# Save the trained ResNet50 model
model.save("/content/drive/My Drive/garbage_classification2/resnet50_model.h5")


Found 15523 files belonging to 2 classes.
['train', 'val'] 2
94765736/94765736 [==============================] - 0s 0us/step
Epoch 1/10
486/486 [==============================] - 2032s 4s/step - loss: 0.6144 - accuracy: 0.7594
Epoch 2/10
486/486 [==============================] - 59s 120ms/step - loss: 0.5080 - accuracy: 0.7940
Epoch 3/10
486/486 [==============================] - 61s 124ms/step - loss: 0.4815 - accuracy: 0.8011
Epoch 4/10
486/486 [==============================] - 64s 130ms/step - loss: 0.4581 - accuracy: 0.8052
Epoch 5/10
486/486 [==============================] - 65s 131ms/step - loss: 0.4308 - accuracy: 0.8142
Epoch 6/10
486/486 [==============================] - 64s 129ms/step - loss: 0.4099 - accuracy: 0.8184
Epoch 7/10
486/486 [==============================] - 64s 130ms/step - loss: 0.3840 - accuracy: 0.8328
Epoch 8/10
486/486 [==============================] - 57s 116ms/step - loss: 0.3586 - accuracy: 0.8466
Epoch 9/10
486/486 [==============================]

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

# Define paths
model_path = "/content/drive/My Drive/garbage_classification2/resnet50_model.h5"
base_image_path = "/content/drive/My Drive/garbage_classification2/train/white-glass/white-glass"

# Load the custom trained ResNet50 model
model = tf.keras.models.load_model(model_path)

# Function to load and preprocess an image
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_preprocessed = tf.keras.applications.resnet50.preprocess_input(img_array)
    return img_preprocessed

# Function to process an image with the model and measure processing time
def process_image_and_time(image_preprocessed, model):
    start_time = time.time()
    prediction = model.predict(image_preprocessed)
    end_time = time.time()
    processing_time = end_time - start_time
    return processing_time

# Process 5 images sequentially from white-glass360.jpg to white-glass365.jpg
for i in range(360, 366):  # Iterate from 360 to 365 (inclusive)
    image_path = f"{base_image_path}{i}.jpg"  # Construct full image path
    img_preprocessed = load_and_preprocess_image(image_path)  # Load and preprocess image
    time_taken = process_image_and_time(img_preprocessed, model)  # Process image and measure time
    print(f"Processing Time for {image_path}: {time_taken:.4f} seconds")


1/1 [==============================] - 1s 1s/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-glass/white-glass360.jpg: 1.4769 seconds
1/1 [==============================] - 0s 203ms/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-glass/white-glass361.jpg: 0.3740 seconds
1/1 [==============================] - 0s 348ms/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-glass/white-glass362.jpg: 0.4394 seconds
1/1 [==============================] - 0s 332ms/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-glass/white-glass363.jpg: 0.7413 seconds
1/1 [==============================] - 0s 203ms/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-glass/white-glass364.jpg: 0.3746 seconds
1/1 [==============================] - 0s 203ms/step
Processing Time for /content/drive/My Drive/garbage_classification2/train/white-